In [1]:
import numpy as np
from math import erf
from scipy import optimize
from scipy import special

C:\Users\00103168\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\00103168\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\00103168\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Input parameters
# Gas constant
R=8.3145
# Reference pressure
Pr=1.5e9
# Reference temperature
TKr=1473.
T0=273.
# Grain size and reference grain size
d=1.e-3
dr=1.e-3
# Reference density
rhor=3176.15
# Thermal expansivity
alphaT=3.59e-5
# Bulk modulus
bmod=115.2
# Raw frequency
freq=0.01

# Brent temperature minimization bounds
AX=0.
CX=2000.
tol=1e-3

# Other parameters (density, compressibility etc.):
rho0=3300.
a0=2.832e-5
a1=0.758e-8
K0=130.e9
KT=4.8
grun=6.
p0=3330.
AY=1.0
CY=1.6

# Set key parameters
Ab=0.664
alpha=0.38
tauP=6.e-5
Teta=0.94
beta=0.
delphi=0.
gamma=5.
lambdaphi=0.

# Set dV0 vs. P parameterisation parameters
y_a = -7.334963115431564676e-23
y_b = 7.510867653681621105e-12
y_c = 1.000184023114681908e+00

# Set solidus temperature at 50 km depth
solidus_50km = 1380
def funcVs(T,Vs_obs,m,dep,Mg):

  # Difference between observed and calculated temperature
  funcVs=np.abs(Vs_obs-Vs_calc(m,T,dep,Mg))
  return funcVs

def funcV0(x,Pin,K0,KT):

  # Difference between observed and calculated pressure for given value of P
  funcV0=np.abs(((K0*(3./2.)*(x**(7./3.)-x**(5./3.))*(1.+(((3./4.)*(KT-4.))*(x**(2./3.)-1.))))*1e-9)-(Pin*1e-9))
  return funcV0


In [3]:
#ALL 1333
pos=np.array([ 7.51212501e+01, -1.28055870e-02,  1.96132180e+00,  2.53289695e+01,
        8.59644885e+05,  2.99627780e-06,  1.14668689e+00, -5.14921992e+00,
       -1.64919976e+03])

In [4]:
# Inport data
tomo_NA_stack = np.loadtxt('ANT-20-400.txt')

In [5]:
def Vs_calc3(m,T,dep,Mg):

  mu0 = m[0]
  dmudT = m[1]
  dmudP = m[2]
  eta0 = 10**m[3]
  E = m[4]
  Va = m[5]
  dTdz = m[6]
  dmudMg = m[7]
  dpdMg = m[8]
  sol50 = 1380

  Pg=(dep/30.)
  P=Pg*1.e9
  TK=T+273.

  Tsol=sol50+(dTdz*(dep-50.))
  Tn=TK/(Tsol+273.)

  Aeta=(1./gamma)*np.exp(-delphi)*np.ones(len(Tn))
  Aeta[Tn < Teta] = 1
  Aeta[(Tn < Teta) & (Tn <1)] = np.exp((-1.*((Tn[(Tn < Teta) & (Tn <1)]-Teta)/(Tn[(Tn < Teta) & (Tn <1)]-(Tn[(Tn < Teta) & (Tn <1)]*Teta))))*np.log(gamma))


    
  # Work out viscosity given A
  eta=((eta0*np.exp((E/R)*(1./TK-1./TKr))*np.exp((Va/R)*(P/TK-Pr/TKr)))*Aeta)

  # Unrelaxed compliance
  Ju=1./(1.e9*(mu0+(dmudP*Pg)+(dmudT*T)+(dmudMg*(100-Mg)/100)))


  # Determine input parameters for complex compliance terms
  tauM=eta*Ju
  tau=(3.*dep)/4.2
  tauS=tau/(2*np.pi*tauM)

  Ap=(0.03+beta)*np.ones(len(Tn))

  Ap [Tn < 0.91]=0.01
  Ap [(Tn>=0.91)&(Tn<0.96)]=0.01+(0.4*(Tn[(Tn>=0.91)&(Tn<0.96)]-0.91))
  Ap [(Tn>=0.96)&(Tn<1)]=0.03

  sigmap=7*np.ones(len(Tn))

  sigmap [Tn < 0.92]=4
  sigmap [(Tn>=0.92)&(Tn<1)]=4.+(37.5*(Tn[(Tn>=0.92)&(Tn<1)]-0.92))

  # Determine complex compliance terms
  J1=Ju*(1.+((Ab*(tauS**alpha))/alpha)+((np.sqrt(2.*np.pi)/2.)*Ap*sigmap*(1.-special.erf((np.log(tauP/tauS))/(np.sqrt(2.)*sigmap)))))

  # include pressure and temperature-dependent alpha
  dV0=optimize.brent(funcV0,brack=(AY,CY),args=(P,K0,KT,),tol=tol)
  alphaP0=dV0*np.exp((grun+1.)*((dV0**(-1.))-1.))
  rhoP0=(p0+dpdMg*(Mg-89.27)/100)*dV0
  intalphaT=(a0*(TK-273.))+((a1/2.)*((TK**2.)-(273.**2.)))
  rho=(rhoP0)*(1.-(alphaP0*intalphaT))

  rhoP01=(3330)*dV0
  rho00=(rhoP01)*(1.-(alphaP0*intalphaT))

  # Calculate Vs
  Vs=1./(np.sqrt(rho*J1)*1000.)  
  TableVRho=np.empty([nT, 4])
  TableVRho[:, 0] = Vs
  TableVRho[:, 1] = rho
  TableVRho[:, 2] = rho00
  TableVRho[:, 3] = eta
  return TableVRho

In [9]:
Tmin = 300-273.
Tmax = 3000-273.
dT = 1.      # temperature increment
nT = int((Tmax - Tmin)/dT + 1)
depth=tomo_NA_stack[:,2]
Depths=np.unique(-depth)
nDepths = Depths.shape[0]

In [10]:
Depths

array([  1000.,   2000.,   3000.,   4000.,   5000.,   6000.,   7000.,
         8000.,   9000.,  10000.,  15000.,  20000.,  25000.,  30000.,
        35000.,  40000.,  45000.,  50000.,  55000.,  60000.,  65000.,
        70000.,  75000.,  80000.,  85000.,  90000.,  95000., 100000.,
       105000., 110000., 115000., 120000., 125000., 130000., 135000.,
       140000., 145000., 150000., 155000., 160000., 165000., 170000.,
       175000., 180000., 185000., 190000., 195000., 200000., 205000.,
       210000., 215000., 220000., 225000., 230000., 235000., 240000.,
       245000., 250000., 255000., 260000., 265000., 270000., 275000.,
       280000., 285000., 290000., 295000., 300000., 305000., 310000.,
       315000., 320000., 325000., 330000., 335000., 340000., 345000.,
       350000., 355000., 360000., 365000., 370000., 375000., 380000.,
       385000., 390000., 395000., 400000.])

In [11]:
arr_T = np.linspace(Tmin, Tmax, nT)
Result_T = list()
Result_Rho = list()
Result_eta = list()
Result_V = list()

for i in range(nDepths):
    arr_select = tomo_NA_stack[:, 2]== -Depths[i]
    obs_v = tomo_NA_stack[:, 3][arr_select]/1e3
    arr_TVRho = np.zeros([nT, 5])
    arr_TVRho[:, 0] = arr_T
    arr_TVRho[:, 1:] = Vs_calc3(pos,arr_T,Depths[i]/1e3,89.27)
    syn_T = arr_TVRho[:, 0]
    syn_v = arr_TVRho[:, 1]
    syn_rho = arr_TVRho[:, 2]  
    syn_eta = arr_TVRho[:, 4]
    ipkwds = dict(left=-1, right=-1)
    result_T = np.interp(obs_v, syn_v[::-1], syn_T[::-1], **ipkwds)
    result_rho = np.interp(obs_v, syn_v[::-1], syn_rho[::-1], **ipkwds)
    result_eta = np.interp(obs_v, syn_v[::-1], syn_eta[::-1], **ipkwds)
    result_v = np.interp(obs_v, syn_v[::-1], syn_v[::-1], **ipkwds)

    Result_V.extend(result_v.tolist())
    Result_T.extend(result_T.tolist())
    Result_Rho.extend(result_rho.tolist())
    Result_eta.extend(result_eta.tolist())

Result_T = np.asarray(Result_T)
Result_eta = np.asarray(Result_eta)
Result_Rho = np.asarray(Result_Rho)
Out = np.zeros([len(Result_Rho), 8])
Out[:, 0:4] = tomo_NA_stack[:,0:4]
Out[:, 4] = Result_T
Out[:, 5] = Result_Rho
Out[:, 6] = Result_eta
Out[:, 7] = Result_V
Outs=Out

In [12]:
np.savetxt('Ant_20_PUM_att_adiabte_F_priorden.txt', Outs, delimiter=",")

In [13]:
arr_T = np.linspace(Tmin, Tmax, nT)
Result_T = list()
Result_Rho = list()
Result_eta = list()
Result_V = list()

for i in range(nDepths):
    arr_select = tomo_NA_stack[:, 2]== -Depths[i]
    obs_v = tomo_NA_stack[:, 3][arr_select]/1e3
    arr_TVRho = np.zeros([nT, 5])
    arr_TVRho[:, 0] = arr_T
    arr_TVRho[:, 1:] = Vs_calc3(pos,arr_T,Depths[i]/1e3,91.27)
    syn_T = arr_TVRho[:, 0]
    syn_v = arr_TVRho[:, 1]
    syn_rho = arr_TVRho[:, 2]  
    syn_eta = arr_TVRho[:, 4]
    ipkwds = dict(left=-1, right=-1)
    result_T = np.interp(obs_v, syn_v[::-1], syn_T[::-1], **ipkwds)
    result_rho = np.interp(obs_v, syn_v[::-1], syn_rho[::-1], **ipkwds)
    result_eta = np.interp(obs_v, syn_v[::-1], syn_eta[::-1], **ipkwds)
    result_v = np.interp(obs_v, syn_v[::-1], syn_v[::-1], **ipkwds)

    Result_V.extend(result_v.tolist())
    Result_T.extend(result_T.tolist())
    Result_Rho.extend(result_rho.tolist())
    Result_eta.extend(result_eta.tolist())

Result_T = np.asarray(Result_T)
Result_eta = np.asarray(Result_eta)
Result_Rho = np.asarray(Result_Rho)
Out = np.zeros([len(Result_Rho), 8])
Out[:, 0:4] = tomo_NA_stack[:,0:4]
Out[:, 4] = Result_T
Out[:, 5] = Result_Rho
Out[:, 6] = Result_eta
Out[:, 7] = Result_V
Outs=Out

In [14]:
np.savetxt('Ant_20_913_att_adiabte_F_priorden.txt', Outs, delimiter=",")